In [2]:
import pandas as pd 
from sqlalchemy import and_
from models import Player, TradPlayerStats, AdvPlayerStats, Game
from data_manager import DataManager
dm = DataManager()
session = dm.get_session()
player_name = "Nikola Jokic"
player = session.query(Player).filter(Player.name==player_name).all()[0]
player_id = player.id

stats = session.query(TradPlayerStats).join(TradPlayerStats.game).filter(TradPlayerStats.player_id == player_id).all()
stats = dm.convert_to_df(stats)
print(stats.columns)


Index(['id', 'fg3a', 'ast', 'player_id', 'fg3_pct', 'stl', 'start_position',
       'ftm', 'blk', 'minutes', 'fta', 'to', 'fgm', 'ft_pct', 'pf', 'fga',
       'oreb', 'pts', 'fg_pct', 'dreb', 'plus_minus', 'game_id', 'fg3m',
       'reb'],
      dtype='object')


In [3]:

# Query to join four tables with correct join conditions and multiple filters
data = session.query(
    Player,
    TradPlayerStats,
    AdvPlayerStats,
    Game
).join(Game, TradPlayerStats.game_id == Game.id)\
 .join(Player, TradPlayerStats.player_id == Player.id)\
 .join(AdvPlayerStats, and_(TradPlayerStats.game_id == AdvPlayerStats.game_id, 
                            TradPlayerStats.player_id == AdvPlayerStats.player_id))\
 .filter(
     (TradPlayerStats.player_id == player_id)# &
    #  (Game.season_type == "Playoffs")
 ).all()

# Convert the query result to a DataFrame
data_list = []
for player, trad_stats, adv_stats, game in data:
    row = {
        'player_name': player.name,
        'player_position': player.position,
        'minutes': trad_stats.minutes,
        'points': trad_stats.pts,
        'efg': adv_stats.efg_pct,
        'date': game.date,
    }
    data_list.append(row)

data_df = pd.DataFrame(data_list)

print(data_df)


     player_name player_position  minutes  points    efg        date
0   Nikola Jokic               C     41.0      25  0.595  2024-04-29
1   Nikola Jokic               C     41.0      33  0.550  2024-04-27
2   Nikola Jokic               C     38.0      24  0.692  2024-04-25
3   Nikola Jokic               C     41.0      27  0.625  2024-04-22
4   Nikola Jokic               C     39.0      32  0.674  2024-04-20
..           ...             ...      ...     ...    ...         ...
83  Nikola Jokic               C     42.0      38  0.500  2023-11-24
84  Nikola Jokic               C     38.0      36  0.538  2023-11-12
85  Nikola Jokic               C     40.0      36  0.646  2024-04-04
86  Nikola Jokic               C     37.0      22  0.313  2023-12-06
87  Nikola Jokic               C     35.0      32  0.391  2023-11-14

[88 rows x 6 columns]


In [4]:
data_df = pd.DataFrame(data_list)
data_df.to_csv('data.csv')